In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import csv

custom_headers = {
    "accept-language": "en-GB,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}
reviews = []
root_url = 'https://www.amazon.com/OneGame-Ergonomic-Adjustable-Backrest-Blackgray/product-reviews/B0CNZ287Y1/ref=cm_cr_getr_d_paging_btm_prev_1?ie=UTF8&pageNumber=1&reviewerType=all_reviews'

In [96]:
def get_soup(url):
    response = requests.get(url, headers=custom_headers)

    if response.status_code != 200:
        print("Error in getting webpage")
        exit(-1)

    soup = BeautifulSoup(response.text, "lxml")
    return soup

In [97]:
def get_date(text):
    pattern = r'on\s([A-Za-z]+\s\d{1,2},\s\d{4})'
    match = re.search(pattern, text)
    return match.group(1)

In [98]:
def get_onepage(soup, reviews):
    review_elements = soup.find_all('div', class_='a-section review aok-relative')
    for review in review_elements:
        username = review.find('div', class_='a-profile-content').text
        date_text = review.find('span', class_='a-size-base a-color-secondary review-date').text
        date = get_date(date_text)
        review_text = review.find('div', class_='a-row a-spacing-small review-data').text
        reviews.append({'username': username, 'date': date, 'review': review_text})

In [99]:
def get_multiple_page(url):    
    for page in range(1, 11):
        soup = get_soup(root_url)
        get_onepage(soup, reviews)
        li = soup.find('li', class_='a-last')
        url = li.find('a')['href']
        print(len(reviews))
        time.sleep(1)
        print(f'page {page} ...')

In [100]:
get_multiple_page(root_url)

10
page 1 ...
20
page 2 ...
30
page 3 ...
40
page 4 ...
50
page 5 ...
60
page 6 ...
70
page 7 ...
80
page 8 ...
90
page 9 ...
100
page 10 ...


In [101]:
with open('reviews.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['username', 'date', 'review'])
    writer.writeheader()
    writer.writerows(reviews)

In [102]:
df = pd.read_csv('reviews.csv')
df

,username,date,review
0,bing,"March 14, 2024",\nI just bought the Onegame gaming chair with ...
1,Quinci,"March 13, 2024",\nThis is a very solid option for affordable g...
2,JZ,"March 1, 2024",\nMy son really likes his new gaming chair. T...
3,Stephanie,"March 15, 2024",\nMy rating is for the instructions that are h...
4,Sherry Dallo,"March 2, 2024",\nThe chairs are great! My husband likes it ve...
...,...,...,...
95,WENDY,"March 1, 2024",\nThe chair is very comfortable. I especially ...
96,Travis L.,"March 11, 2024",\nI think looks and functionality it's a 10/10...
97,kiwi,"March 11, 2024","\nThe quality is very good, very easy to insta..."
98,Lau C.,"March 5, 2024",\nThe chair’s manual was short and easy to und...
